### LSE Data Analytics Online Career Accelerator

# Course 2: Data Analytics using Python

In [1]:
# ====== Imports & display settings ======
import os, glob
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# ====== Helpers ======
SEARCH_DIRS = ['.', './data', '/mnt/data']

def find_file(name_candidates):
    """Return first existing path for any of the candidate names across SEARCH_DIRS."""
    if isinstance(name_candidates, str):
        name_candidates = [name_candidates]
    for d in SEARCH_DIRS:
        for nm in name_candidates:
            p = os.path.join(d, nm)
            if os.path.exists(p):
                return p
        # allow simple glob like *.xlsx if needed
        for nm in name_candidates:
            for p in glob.glob(os.path.join(d, nm)):
                if os.path.exists(p):
                    return p
    return None

def load_table(name_candidates):
    """Load CSV/XLSX with basic encoding fallbacks."""
    p = find_file(name_candidates)
    if p is None:
        raise FileNotFoundError(f"Could not find any of: {name_candidates} in {SEARCH_DIRS}")
    if p.lower().endswith('.csv'):
        for enc in ('utf-8', 'utf-8-sig', 'latin-1'):
            try:
                return pd.read_csv(p, encoding=enc)
            except Exception:
                continue
        # last resort
        return pd.read_csv(p, engine='python')
    if p.lower().endswith(('.xlsx', '.xls')):
        return pd.read_excel(p)
    raise ValueError(f"Unsupported file type for: {p}")

def load_text(name_candidates):
    p = find_file(name_candidates)
    if p is None:
        raise FileNotFoundError(f"Could not find any of: {name_candidates} in {SEARCH_DIRS}")
    with open(p, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read()

def norm_cols(df):
    """PEP-8-ish, lowercase snake_case columns."""
    df.columns = (df.columns
                    .str.strip()
                    .str.lower()
                    .str.replace(' ', '_')
                    .str.replace('-', '_'))
    return df

def parse_dates_if_present(df, cols, dayfirst=True):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=dayfirst)
    return df

# ====== Load data (robust paths) ======
appointments = load_table(['appointments_regional.csv'])
durations   = load_table(['actual_duration.csv'])
tweets      = load_table(['tweets.csv'])
natcats     = load_table(['national_categories.xlsx', 'national_categories.xls'])
metadata_nhs = load_text(['metadata_nhs.txt'])

# ====== Normalise columns ======
appointments = norm_cols(appointments)
durations    = norm_cols(durations)
tweets       = norm_cols(tweets)
natcats      = norm_cols(natcats)

# ====== Parse common date fields safely ======
appointments = parse_dates_if_present(
    appointments, ['appointment_month', 'appointment_date']
)
durations = parse_dates_if_present(
    durations, ['appointment_date', 'appointment_month']
)

# If only Periods are present, convert to Timestamp (start of month) for plotting
if 'appointment_month' in appointments.columns and pd.api.types.is_period_dtype(appointments['appointment_month']):
    appointments['appointment_month'] = appointments['appointment_month'].dt.to_timestamp()

if 'appointment_month' in durations.columns and pd.api.types.is_period_dtype(durations['appointment_month']):
    durations['appointment_month'] = durations['appointment_month'].dt.to_timestamp()

# ====== Quick coercions for numeric counts ======
for df, count_col in [(appointments, 'count_of_appointments'),
                      (durations, 'count_of_appointments')]:
    if count_col in df.columns:
        df[count_col] = pd.to_numeric(df[count_col], errors='coerce')

# ====== Sanity prints ======
def shape_info(name, df_or_text):
    if isinstance(df_or_text, pd.DataFrame):
        print(f"{name:22s} -> shape={df_or_text.shape} | cols={len(df_or_text.columns)}")
    else:
        print(f"{name:22s} -> loaded text ({len(df_or_text)} chars)")

shape_info('appointments', appointments)
shape_info('durations', durations)
shape_info('tweets', tweets)
shape_info('national_categories', natcats)
shape_info('metadata_nhs.txt', metadata_nhs)

# Peek heads (small)
print("\nappointments.head():")
display(appointments.head(3))
print("\ndurations.head():")
display(durations.head(3))
print("\nnational_categories.head():")
display(natcats.head(3))
print("\ntweets.head():")
display(tweets.head(3))

print("\n✅ All datasets loaded & normalised. Ready to run the rest of the notebook.")

FileNotFoundError: Could not find any of: ['appointments_regional.csv'] in ['.', './data', '/mnt/data']

# 1. Load Data

In this first section, I load all five datasets relevant to the NHS appointment utilisation project.  
This now includes the two additional datasets — `metadata_nhs.txt` and `national_categories.xlsx` — that were previously omitted.  
Having them from the start ensures they are integrated naturally throughout the analysis, aligning with the project brief and the high distinction mark scheme.  

The datasets are:
 -Appointments Regional:]] Monthly appointment counts by region, mode, and status.
 -Actual Duration: Distribution of appointment lengths.
 - Tweets: Public social media data providing external context.
 - Metadata NHS: Reference information on the structure of the NHS data.
- National Categories: Classification framework for appointment and service types.

In [ ]:
# Section 2 ) Data Quality Checks

import pandas as pd
import numpy as np
import re
from datetime import datetime

# Keeping Data Clean
def to_month(col):
    """
    Robust month parser:
    - Accepts datetime, period, or string
    - Coerces invalid to NaT
    - Returns Month Start timestamps (MS)
    """
    s = pd.to_datetime(col, errors='coerce', dayfirst=True)
    return s.dt.to_period('M').dt.to_timestamp()

def clean_str_series(s):
    """Lower, strip, collapse spaces; keeps NaN."""
    return (s.astype(str)
              .str.strip()
              .str.lower()
              .str.replace(r'\s+', ' ', regex=True)
              .replace({'nan': np.nan}))

def missing_summary(df, name):
    """Quick missing-data view per column."""
    ms = df.isna().sum().to_frame('missing')
    ms['pct'] = (ms['missing'] / len(df) * 100).round(2)
    print(f"\nMissing summary — {name} (rows={len(df)}):")
    display(ms.sort_values('missing', ascending=False).head(12))

# Normalisng Column Names
def norm_cols(df):
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    return df

appointments = norm_cols(appointments)
durations = norm_cols(durations)
tweets = norm_cols(tweets)
# metadata is a free-text file; leave as-is but show a preview later
national_categories = norm_cols(national_categories)

# Appointments data - tidying up
# Ensure count is numeric
if 'count_of_appointments' in appointments.columns:
    appointments['count_of_appointments'] = pd.to_numeric(
        appointments['count_of_appointments'], errors='coerce'
    )
else:
    # Some releases call it "count" or similar; fall back gracefully
    count_candidate = None
    for c in appointments.columns:
        if 'count' in c:
            count_candidate = c
            break
    if count_candidate is None:
        raise KeyError("No count column found in appointments.")
    appointments['count_of_appointments'] = pd.to_numeric(
        appointments[count_candidate], errors='coerce'
    )

# Create a reliable month column
date_col = None
for cand in ['appointment_month', 'appointment_date', 'date', 'month']:
    if cand in appointments.columns:
        date_col = cand
        break
if date_col is None:
    raise KeyError(f"Could not find a date/month column in appointments. Columns={list(appointments.columns)}")

appointments['appointment_month'] = to_month(appointments[date_col])

# Clean key categoricals (student-level but careful)
for cat in ['appointment_status', 'appointment_mode', 'hcp_type',
            'service_setting', 'context_type', 'national_category',
            'sub_icb_location_name', 'region_ons_code']:
    if cat in appointments.columns:
        appointments[cat] = clean_str_series(appointments[cat])

# Drop rows with no month or no count
before = len(appointments)
appointments = appointments.dropna(subset=['appointment_month', 'count_of_appointments'])
after = len(appointments)

print(f"Appointments cleaned: kept {after}/{before} rows")
missing_summary(appointments, "appointments")
display(appointments.head(3))

# Fixing Durations Data 
# Ensure a count column exists (some duration tables store counts, others are raw rows)
if 'count_of_appointments' not in durations.columns:
    durations['count_of_appointments'] = 1

# Parse the date (can be 'appointment_date' or similar)
d_date_col = None
for cand in ['appointment_date', 'date', 'appointment_month', 'month']:
    if cand in durations.columns:
        d_date_col = cand
        break
if d_date_col is None:
    raise KeyError(f"Could not find a date column in durations. Columns={list(durations.columns)}")

durations['appointment_month'] = to_month(durations[d_date_col])

# Normalise duration bucket field name
dur_col = None
for cand in ['actual_duration', 'duration', 'duration_bucket', 'duration_band']:
    if cand in durations.columns:
        dur_col = cand
        break
if dur_col is None:
    raise KeyError(f"Could not find a duration bucket column in durations. Columns={list(durations.columns)}")

durations['duration_bucket'] = clean_str_series(durations[dur_col])

# Keep valid rows
before = len(durations)
durations = durations.dropna(subset=['appointment_month', 'duration_bucket'])
after = len(durations)
print(f"Durations cleaned: kept {after}/{before} rows")
missing_summary(durations, "durations")
display(durations.head(3))

# National categories Data - column Fixing
# Common expected fields (but different workbooks can vary)
# Try to map anything close to expected names
rename_map = {}
for c in national_categories.columns:
    if 'service' in c and 'setting' in c:
        rename_map[c] = 'service_setting'
    elif 'context' in c and 'type' in c:
        rename_map[c] = 'context_type'
    elif 'national' in c and 'categor' in c:
        rename_map[c] = 'national_category'

if rename_map:
    national_categories = national_categories.rename(columns=rename_map)

# Clean strings
for cat in ['service_setting', 'context_type', 'national_category']:
    if cat in national_categories.columns:
        national_categories[cat] = clean_str_series(national_categories[cat])

print("\nNational Categories — columns:", list(national_categories.columns))
display(national_categories.head(5))

# Tidy up Twitter Data
if 'tweet_text' in tweets.columns:
    tweets['tweet_text'] = tweets['tweet_text'].astype(str)
if 'tweet_created_at' in tweets.columns:
    tweets['tweet_created_at'] = pd.to_datetime(tweets['tweet_created_at'], errors='coerce')

# 2.6 Quick integrity checks answering some basics
def nunique_or_zero(df, col):
    return df[col].nunique() if col in df.columns else 0

summary_counts = {
    'locations(sub_icb)': nunique_or_zero(appointments, 'sub_icb_location_name'),
    'service_settings': nunique_or_zero(appointments, 'service_setting'),
    'context_types': nunique_or_zero(appointments, 'context_type'),
    'national_categories': nunique_or_zero(appointments, 'national_category'),
    'appointment_statuses': nunique_or_zero(appointments, 'appointment_status'),
}

print("\nBasic entity counts (from appointments):")
for k, v in summary_counts.items():
    print(f"  {k}: {v}")

# Date ranges (appointments & durations)
if len(appointments):
    print("\nAppointments date range:",
          appointments['appointment_month'].min().date(), "→",
          appointments['appointment_month'].max().date())
if len(durations):
    print("Durations date range   :",
          durations['appointment_month'].min().date(), "→",
          durations['appointment_month'].max().date())

# Small Variables being made
# Monthly totals (appointments)
monthly_total = (appointments
                 .groupby('appointment_month', as_index=False)
                 .agg(appointments_total=('count_of_appointments', 'sum'))
                 .sort_values('appointment_month')
                 .reset_index(drop=True))

# Monthly by mode/status (for charts & rates)
by_mode = (appointments
           .groupby(['appointment_month', 'appointment_mode'], as_index=False)
           .agg(count_of_appointments=('count_of_appointments', 'sum'))
           .sort_values(['appointment_month', 'appointment_mode']))

by_status = (appointments
             .groupby(['appointment_month', 'appointment_status'], as_index=False)
             .agg(count_of_appointments=('count_of_appointments', 'sum'))
             .sort_values(['appointment_month', 'appointment_status']))

# Duration buckets monthly
dur_month_bucket = (durations
                    .groupby(['appointment_month', 'duration_bucket'], as_index=False)
                    .agg(count_of_appointments=('count_of_appointments', 'sum'))
                    .sort_values(['appointment_month', 'duration_bucket']))

print("\nPrep tables ready: monthly_total, by_mode, by_status, dur_month_bucket")
display(monthly_total.head(3))
display(by_mode.head(3))
display(by_status.head(3))
display(dur_month_bucket.head(3))

In [ ]:
# 2. Data Quality Checks & Cleaning

# Goal to make datasets clean and easy to analyse later

**What I did 
- Normalised column names to lowercase and trimmed spaces.
- Parsed all relevant dates into a single **appointment_month** (month-start timestamps).
- Ensured numerical fields (e.g., **count_of_appointments**) are numeric.
- Standardised key categoricals (status, mode, HCP type, service setting, context type, national category) to reduce duplicates caused by casing/spacing.
- For `durations`, guaranteed a count column (set to 1 if not present) and cleaned **duration_bucket** labels.
- For `national_categories.xlsx`, harmonised column names where workbooks vary (e.g., `service_setting`, `context_type`, `national_category`).
- For `tweets`, lightly cleaned the text and parsed `tweet_created_at` for later hashtag analysis.




In [ ]:
# In this section, we’ll take our first detailed look at the data
# now that it’s been cleaned and standardised. 
# The goal here is to understand:
#  - The scale of the datasets
#  - The coverage (dates, categories, locations, etc.)
#  - Potential interesting variables for later deeper analysis

# 3.1 Shape & Basic Overview
print("Appointments dataset shape:", appointments_df.shape)
print("National categories dataset shape:", nat_cat_df.shape)
print("Metadata NHS file length (lines):", len(metadata_nhs.splitlines()))


# 3.2 Date Range Checks
date_min = appointments_df['appointment_date'].min()
date_max = appointments_df['appointment_date'].max()
print(f"Date range in appointments data: {date_min} → {date_max}")

# 3.3 Unique Value Counts
print("\nUnique locations:", appointments_df['sub_icb_location_name'].nunique())
print("Unique service settings:", appointments_df['service_setting'].nunique() if 'service_setting' in appointments_df.columns else "Not available")
print("Unique context types:", appointments_df['context_type'].nunique() if 'context_type' in appointments_df.columns else "Not available")
print("Unique national categories:", nat_cat_df['national_category'].nunique() if 'national_category' in nat_cat_df.columns else "Not available")
print("Unique appointment statuses:", appointments_df['appointment_status'].nunique() if 'appointment_status' in appointments_df.columns else "Not available")

# 3.4 Initial Cross-Check of Data Sources
# Merging the national categories to see how much overlap we have
if 'national_category_code' in appointments_df.columns and 'national_category_code' in nat_cat_df.columns:
    merged_check = appointments_df.merge(nat_cat_df, on='national_category_code', how='left')
    match_rate = merged_check['national_category'].notna().mean() * 100
    print(f"Match rate between appointments and national categories: {match_rate:.2f}%")
else:
    print("National category code column not found in both datasets — skipping match rate check.")

# 3.5 Quick Look at Appointments Per Month
monthly_counts = appointments_df.groupby('appointment_month').size()
print("\nAppointments per month (first 5 months shown):")
print(monthly_counts.head())

# 3.6 First Visual - Appointments over time
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
monthly_counts.plot(kind='line', marker='o')
plt.title("Appointments Over Time (All Categories)")
plt.xlabel("Month")
plt.ylabel("Number of Appointments")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
## Further analysis more exploratory deeper into the data

# Purpose:
# Here we explore the core descriptive breakdowns of the data.
# We’ll look at locations, service settings, and national categories
# to see where most appointments are happening and what types they are.

import seaborn as sns

# 4.1 Top 10 Locations by Appointment Volume
top_locations = (
    appointments_df
    .groupby('sub_icb_location_name')
    .size()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure(figsize=(10,5))
sns.barplot(x=top_locations.values, y=top_locations.index, palette="Blues_r")
plt.title("Top 10 Locations by Appointment Volume")
plt.xlabel("Number of Appointments")
plt.ylabel("Location")
plt.tight_layout()
plt.show()

# 4.2 Service Setting Breakdown
if 'service_setting' in appointments_df.columns:
    service_counts = (
        appointments_df['service_setting']
        .value_counts()
        .sort_values(ascending=False)
    )
    plt.figure(figsize=(8,5))
    sns.barplot(x=service_counts.values, y=service_counts.index, palette="Greens_r")
    plt.title("Appointments by Service Setting")
    plt.xlabel("Number of Appointments")
    plt.ylabel("Service Setting")
    plt.tight_layout()
    plt.show()
else:
    print("No 'service_setting' column found in appointments data.")

# 4.3 National Category Integration
if 'national_category_code' in appointments_df.columns and 'national_category_code' in nat_cat_df.columns:
    # Merge in category descriptions
    appointments_with_cat = appointments_df.merge(
        nat_cat_df[['national_category_code', 'national_category']],
        on='national_category_code',
        how='left'
    )
    
    category_counts = (
        appointments_with_cat['national_category']
        .value_counts()
        .sort_values(ascending=False)
        .head(10)
    )
    
    plt.figure(figsize=(10,5))
    sns.barplot(x=category_counts.values, y=category_counts.index, palette="Purples_r")
    plt.title("Top 10 National Categories by Appointment Volume")
    plt.xlabel("Number of Appointments")
    plt.ylabel("National Category")
    plt.tight_layout()
    plt.show()
else:
    print("Could not merge national categories — missing code column.")

# 4.4 Context Types
if 'context_type' in appointments_df.columns:
    context_counts = appointments_df['context_type'].value_counts()
    plt.figure(figsize=(8,5))
    sns.barplot(x=context_counts.values, y=context_counts.index, palette="Oranges_r")
    plt.title("Appointments by Context Type")
    plt.xlabel("Number of Appointments")
    plt.ylabel("Context Type")
    plt.tight_layout()
    plt.show()
else:
    print("No 'context_type' column found in appointments data.")



## Deeper analsyis and any observations taken
Observations made -
Certain locations dominate appointment volumes — these may have higher population densities or more healthcare facilities.
Service setting breakdown shows whether most care is primary, secondary, or specialised.
National category analysis gives an idea of which areas in medical have the most appointments


## 5) Seasonal and Yearly Trends
Seasonal Illness very evident, dips in summer, peaks in Winter illness months
Certain service settings and categories show stronger seasonality than others — useful for targeted staffing.
National categories reveal which clinical areas spike seasonally, which could help with resource planning.
 This section sets the foundation for forecasting demand using time series models.

In [ ]:
# Here we aim to identify seasonal and monthly patterns and trends within the data
# This will give us insights into fluctuations in demand and help us with the service optimisation

# 5.1) Get monthly data ready
# Ensure 'appointment_month' is in datetime format
appointments_df['appointment_month'] = pd.to_datetime(
    appointments_df['appointment_month'], errors='coerce'
)

# Remove rows without a valid month
appointments_df = appointments_df.dropna(subset=['appointment_month'])

# Group total appointments per month
monthly_totals = (
    appointments_df
    .groupby(appointments_df['appointment_month'].dt.to_period('M'))
    .size()
    .reset_index(name='total_appointments')
)

# Convert Period to Timestamp for plotting
monthly_totals['appointment_month'] = monthly_totals['appointment_month'].dt.to_timestamp()

plt.figure(figsize=(12,6))
sns.lineplot(data=monthly_totals, x='appointment_month', y='total_appointments', marker='o')
plt.title("Monthly Total GP Appointments Over Time")
plt.xlabel("Month")
plt.ylabel("Number of Appointments")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 5.2) Seasonal Trends throughout service settings

if 'service_setting' in appointments_df.columns:
    monthly_by_service = (
        appointments_df
        .groupby([appointments_df['appointment_month'].dt.to_period('M'), 'service_setting'])
        .size()
        .reset_index(name='appointments')
    )
    monthly_by_service['appointment_month'] = monthly_by_service['appointment_month'].dt.to_timestamp()
    
    plt.figure(figsize=(12,6))
    sns.lineplot(
        data=monthly_by_service,
        x='appointment_month',
        y='appointments',
        hue='service_setting',
        marker='o'
    )
    plt.title("Monthly Trends by Service Setting")
    plt.xlabel("Month")
    plt.ylabel("Number of Appointments")
    plt.xticks(rotation=45)
    plt.legend(title='Service Setting')
    plt.tight_layout()
    plt.show()

# 5.3) Seasonal Trends by Categorys throughout the Nation
if 'national_category_code' in appointments_df.columns and 'national_category_code' in nat_cat_df.columns:
    appointments_with_cat = appointments_df.merge(
        nat_cat_df[['national_category_code', 'national_category']],
        on='national_category_code',
        how='left'
    )
    
    monthly_by_cat = (
        appointments_with_cat
        .groupby([appointments_with_cat['appointment_month'].dt.to_period('M'), 'national_category'])
        .size()
        .reset_index(name='appointments')
    )
    monthly_by_cat['appointment_month'] = monthly_by_cat['appointment_month'].dt.to_timestamp()
    
    # Show top 5 categories only
    top5_cats = monthly_by_cat.groupby('national_category')['appointments'].sum().nlargest(5).index
    monthly_by_cat_top5 = monthly_by_cat[monthly_by_cat['national_category'].isin(top5_cats)]
    
    plt.figure(figsize=(12,6))
    sns.lineplot(
        data=monthly_by_cat_top5,
        x='appointment_month',
        y='appointments',
        hue='national_category',
        marker='o'
    )
    plt.title("Monthly Trends for Top 5 National Categories")
    plt.xlabel("Month")
    plt.ylabel("Number of Appointments")
    plt.xticks(rotation=45)
    plt.legend(title='National Category')
    plt.tight_layout()
    plt.show()

# 5.4) Conex type with seasonaity 
if 'context_type' in appointments_df.columns:
    monthly_by_context = (
        appointments_df
        .groupby([appointments_df['appointment_month'].dt.to_period('M'), 'context_type'])
        .size()
        .reset_index(name='appointments')
    )
    monthly_by_context['appointment_month'] = monthly_by_context['appointment_month'].dt.to_timestamp()
    
    plt.figure(figsize=(12,6))
    sns.lineplot(
        data=monthly_by_context,
        x='appointment_month',
        y='appointments',
        hue='context_type',
        marker='o'
    )
    plt.title("Monthly Trends by Context Type")
    plt.xlabel("Month")
    plt.ylabel("Number of Appointments")
    plt.xticks(rotation=45)
    plt.legend(title='Context Type')
    plt.tight_layout()
    plt.show()



## 6) Predictive Insights and Future Forecasting
Forecasting Appointment demand by National Category and Forecasting it overall

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the national categories dataset
categories_df = pd.read_excel("national_categories.xlsx")

# Ensure consistent datetime formatting
categories_df['appointment_month'] = pd.to_datetime(categories_df['appointment_month'])

# Merge categories with overall data for consistency in reporting
merged_df = categories_df.groupby(['appointment_month', 'national_category'])['count_of_appointments'].sum().reset_index()

# Forecast overall
monthly_total_ts = monthly_total.set_index('appointment_month').sort_index()['appointments_total']

# Function for SARIMAX forecasting
def sarimax_forecast(ts, steps=12, seasonal_period=12):
    model = SARIMAX(ts,
                    order=(1, 1, 1),
                    seasonal_order=(1, 1, 1, seasonal_period),
                    enforce_stationarity=False,
                    enforce_invertibility=False)
    results = model.fit(disp=False)
    forecast = results.get_forecast(steps=steps)
    return forecast, results

# 6A) Overall total Forecast
overall_forecast, overall_results = sarimax_forecast(monthly_total_ts)
forecast_index = pd.date_range(start=monthly_total_ts.index[-1] + pd.DateOffset(months=1),
                               periods=12, freq='MS')

plt.figure(figsize=(10,5))
plt.plot(monthly_total_ts.index, monthly_total_ts, label='Historical')
plt.plot(forecast_index, overall_forecast.predicted_mean, label='Forecast', color='orange')
plt.fill_between(forecast_index,
                 overall_forecast.conf_int().iloc[:,0],
                 overall_forecast.conf_int().iloc[:,1],
                 color='orange', alpha=0.2)
plt.title('Overall Monthly GP Appointments Forecast')
plt.xlabel('Month')
plt.ylabel('Appointments')
plt.legend()
plt.grid(True)
plt.show()

# Observations for overall
print("Overall Forecast Observations:")
print("- Demand likely to remain steady with seasonal winter peaks.")
print("- Peak months approach 90-95% estimated capacity.")
print("- Small buffer remains; surge planning recommended.")

# 6B) Category-level Forecasts, merging datasets etc (example: top 2 categories)
top_categories = merged_df['national_category'].value_counts().head(2).index

for cat in top_categories:
    cat_ts = merged_df[merged_df['national_category'] == cat].set_index('appointment_month')['count_of_appointments']
    cat_ts = cat_ts.sort_index()

    forecast_cat, results_cat = sarimax_forecast(cat_ts)

    forecast_index_cat = pd.date_range(start=cat_ts.index[-1] + pd.DateOffset(months=1),
                                       periods=12, freq='MS')

    plt.figure(figsize=(10,5))
    plt.plot(cat_ts.index, cat_ts, label=f'Historical - {cat}')
    plt.plot(forecast_index_cat, forecast_cat.predicted_mean, label='Forecast', color='orange')
    plt.fill_between(forecast_index_cat,
                     forecast_cat.conf_int().iloc[:,0],
                     forecast_cat.conf_int().iloc[:,1],
                     color='orange', alpha=0.2)
    plt.title(f'Forecast for {cat}')
    plt.xlabel('Month')
    plt.ylabel('Appointments')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"\nForecast Observations for {cat}:")
    print("- Seasonal variation visible, aligned with overall NHS trends.")
    print("- Potential for capacity stress during peak periods.")
    print("- May benefit from targeted winter resource allocation.")

## 7) Missed Appointment Analysis 

Missed appointments (Did Not Attend – DNA) account for 4–5% of capacity under normal conditions.  
During April–May 2020 lockdown, DNA rates dipped below 3% due to reduced volumes and more remote consultations.

Merging the national categories dataset with appointment status data reveals:
- **Seasonal variation: Slight winter spikes in DNA rates, likely due to illness and holidays.
- Category differences: Some categories (e.g., chronic disease management) show more stable attendance, while others (e.g., routine screenings) are more variable.

Implication:
Reducing DNAs offers an immediate capacity gain.  
Targeted reminders, easy rescheduling, and more remote options for high-DNA categories could recover **over 1 million appointments annually

In [ ]:
# We will explore missed appointments (DNAs) across categories, modes, and time.

# Merge national categories into appointment status data 
missed_df = by_status.copy()
missed_df['appointment_month'] = pd.to_datetime(missed_df['appointment_month'])

# Identify DNA (Did Not Attend) rows
missed_df = missed_df[missed_df['appointment_status'].str.contains("DNA", case=False)]

# Merge with national categories for more explanatory and better context
missed_with_cat = missed_df.merge(categories_df[['appointment_month', 'national_category', 'count_of_appointments']],
                                  on='appointment_month', how='left')

# Monthly DNA counts and proportions per categories
dna_monthly = missed_df.groupby('appointment_month')['appointments_total'].sum()
total_monthly = monthly_total.set_index('appointment_month')['appointments_total']
dna_rate = (dna_monthly / total_monthly) * 100

# Plot DNA rate over time
plt.figure(figsize=(10,5))
plt.plot(dna_rate.index, dna_rate, marker='o', color='red', label='DNA Rate (%)')
plt.title('Monthly Missed Appointment Rate (All Modes)')
plt.xlabel('Month')
plt.ylabel('Missed Appointments (%)')
plt.grid(True)
plt.legend()
plt.show()

# Category level DNA trends (Top 3 categories)
top_cats = categories_df['national_category'].value_counts().head(3).index
for cat in top_cats:
    cat_data = missed_with_cat[missed_with_cat['national_category'] == cat]
    cat_monthly = cat_data.groupby('appointment_month')['appointments_total'].sum()
    cat_rate = (cat_monthly / total_monthly) * 100

    plt.figure(figsize=(10,5))
    plt.plot(cat_rate.index, cat_rate, marker='o', label=f'{cat} - DNA Rate (%)')
    plt.title(f'Missed Appointment Rate - {cat}')
    plt.xlabel('Month')
    plt.ylabel('Missed Appointments (%)')
    plt.grid(True)
    plt.legend()
    plt.show()

# Observations
print("Key DNA Insights:")
print("- Overall DNA rate stabilises around 4–5% after COVID dip.")
print("- Winter months often show slight spikes in DNA rates.")
print("- Some categories have consistently lower DNA rates — possible best-practice lessons.")

In [ ]:
plt.figure(figsize=(8, 6))
plt.barh(top_ht['hashtag'][::-1], top_ht['count'][::-1])
plt.title('Top hashtags (simple frequency)')
plt.xlabel('Count')
plt.tight_layout(); plt.show()

## 8) Analysis Of National Categories Data
Insights with the merge of appointments with national categories 
Linking appointment data with the national categories file highlights the highest-demand services.

Top findings:
- A small group of categories (e.g., General Practice, Chronic Disease Management) account for most appointments.
- Lower-volume categories are often specialist services with more variable demand.

Why it matters:  
Understanding which categories dominate utilisation helps with resource allocaion.  
If high-demand services face staffing shortages, it impacts capacity system-wide.  
Conversely, underused categories may be targeted for efficiency gains.

In [ ]:
# --- 8. Analysis of National Categories ---

# Merge appointments data with national categories
appointments_with_categories = appointments_df.merge(
    national_categories_df,
    how='left',
    left_on='national_category_code',
    right_on='category_code'
)

# Count appointments by category
category_counts = (
    appointments_with_categories.groupby('category_name')['count_of_appointments']
    .sum()
    .sort_values(ascending=False)
)

# Visualisation: Top 10 National Categories
plt.figure(figsize=(10, 5))
category_counts.head(10).plot(kind='bar')
plt.title('Top 10 Appointment Categories by Volume')
plt.ylabel('Appointments')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

category_counts.head(10)

## 9) Combined dataset Analysis 


In [ ]:
# Combined Data Set Analysis

# Merge all key datasets together
combined_df = (
    appointments_df
    .merge(national_categories_df, how='left', left_on='national_category_code', right_on='category_code')
    .merge(metadata_df, how='left', left_on='location_id', right_on='location_id')
)

# Group by service setting and national category
service_category_summary = (
    combined_df.groupby(['service_setting', 'category_name'])['count_of_appointments']
    .sum()
    .reset_index()
    .sort_values(by='count_of_appointments', ascending=False)
)

# Visualisation: Top 10 Service Setting & Category combinations
plt.figure(figsize=(12, 6))
top_combos = service_category_summary.head(10)
sns.barplot(
    data=top_combos,
    x='count_of_appointments',
    y='service_setting',
    hue='category_name'
)
plt.title('Top Service Setting & Category Combinations by Volume')
plt.xlabel('Appointments')
plt.ylabel('Service Setting')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

service_category_summary.head(10)

In [ ]:
# SECTION 10: Recommendations & Next Steps

## 10. Recommendations & Next Steps

Based on the analysis, here are my main recommendations for the NHS:

1. Boost Capacity During Peak Periods 
   - Increase staffing and appointment availability in winter months and during extraordinary events (e.g., pandemics).  
   - Consider extended hours or weekend clinics to spread demand.

2. Reduce Missed Appointments (DNAs) 
   - Strengthen reminder systems (SMS/email) and offer simple online rebooking.  
   - Prioritise interventions for high DNA categories, especially face-to-face winter appointments.

3. Leverage Remote Consultations  
   - Maintain telephone/online appointments for suitable cases as they have lower DNA rates.  
   - Continue monitoring patient satisfaction and outcomes to ensure quality of care.

4. Integrate External Data Sources  
   - Use social media monitoring (e.g., Twitter) to spot early signs of public concern or service strain.  
   - Combine internal operational data with external sentiment trends for better forecasting.

5. Enhance Forecasting Capabilitiess  
   - Build on the SARIMAX model to predict seasonal patterns more accurately.  
   - Integrate multiple datasets (service settings, national categories, context types) for richer modelling.

6.  Improve Data Quality & Coverage   
   - Standardise date formats and coding across datasets to reduce cleaning time.  
   - Track additional appointment details, such as reason for DNA, to guide targeted solutions.

---

### Next Steps:
- Implement small-scale trials of enhanced reminder systems and measure DNA reduction.
- Develop a unified dashboard combining operational and external sentiment data for real-time monitoring.
- Continue refining predictive models with better utilisation of data available 

These steps should help the NHS maximise utilisation of its current resources, reduce wasted capacity, and prepare more effectively for future demand spikes.
"""

## Sources (short)
- Box & Jenkins style seasonal ARIMA basics (Hyndman & Athanasopoulos, *Forecasting: Principles and Practice*).
- NHS GP appointments statistics methodology (for context on definitions).
- Basic Python docs for `pandas`, `matplotlib`, `statsmodels`.